In [33]:
import pandas as pd
import numpy as np
from haversine import haversine

# 서울 버스정류장 근처 역 개수 세기

In [2]:
train = pd.read_csv("./서울시 지하철역 연계 지하도 공간정보.csv", encoding='cp949')
bus = pd.read_csv("./서울시버스정류소좌표데이터(20220329).csv", encoding='cp949')

### 지하철역이랑 위도경도 하나씩 매칭

In [3]:
train = train.iloc[:, [1, 4, 14, 15]]
train.head()

,노드 WKT,링크 WKT,지하철역코드,지하철역명
0,NaN,LINESTRING(127.00191146253691 37.5711642556191...,266,종로5가
1,NaN,LINESTRING(126.99214130747008 37.5704173818590...,265,종로3가
2,NaN,LINESTRING(127.00193818725627 37.5708237971757...,266,종로5가
3,POINT(127.00167204751554 37.57080494000384),NaN,266,종로5가
4,POINT(127.00174312956021 37.57067876388195),NaN,266,종로5가


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6564 entries, 0 to 6563
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   노드 WKT  3492 non-null   object
 1   링크 WKT  3072 non-null   object
 2   지하철역코드  6564 non-null   int64 
 3   지하철역명   6564 non-null   object
dtypes: int64(1), object(3)
memory usage: 205.2+ KB


In [5]:
#필요없는 부분 잘라내기
train['노드 WKT'] = train['노드 WKT'].str.slice(start=6, stop=-2)
train['링크 WKT'] = train['링크 WKT'].str.slice(start=11, stop=46)

In [6]:
train.head()

,노드 WKT,링크 WKT,지하철역코드,지하철역명
0,NaN,127.00191146253691 37.5711642556191,266,종로5가
1,NaN,126.99214130747008 37.5704173818590,265,종로3가
2,NaN,127.00193818725627 37.5708237971757,266,종로5가
3,127.00167204751554 37.5708049400038,NaN,266,종로5가
4,127.00174312956021 37.5706787638819,NaN,266,종로5가


### 노드값을 가지는 역 처리

In [7]:
#노드값을 가지는 역 따로 분리
train_node = train[train['노드 WKT'].notnull()]
#train_node.info()
train_node.drop('링크 WKT', axis=1, inplace=True)
train_node.head()

C:\Users\User\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,노드 WKT,지하철역코드,지하철역명
3,127.00167204751554 37.5708049400038,266,종로5가
4,127.00174312956021 37.5706787638819,266,종로5가
5,126.977503872108 37.5707211873125,274,광화문(세종문화회관)
6,127.00213359041985 37.5706882472436,266,종로5가
7,126.97321585505145 37.5756486564555,271,경복궁(정부서울청사)


In [8]:
#경도 위도 나누기
tt = train_node['노드 WKT'].str.split(' ', expand=True)
tt.columns = ['경도', '위도']
tt = tt.astype(np.float)
tt.head()

,경도,위도
3,127.001672,37.570805
4,127.001743,37.570679
5,126.977504,37.570721
6,127.002134,37.570688
7,126.973216,37.575649


In [9]:
#합치기
ttt = pd.concat([train_node[['지하철역코드', '지하철역명']], tt], axis=1)

#unique한 값 개수 확인
print(ttt.nunique())

# 한 지하철 역에 여러개의 네트워크 기기가 설치된 곳이 많음, 위도경도 통일 필요 >> 평균 사용
tttt = ttt.groupby(by = '지하철역코드').mean()

#merge
t_node_final = pd.merge(right= train_node[['지하철역코드', '지하철역명']], left=tttt, how = 'left', on='지하철역코드')
t_node_final = t_node_final.drop_duplicates()
t_node_final = t_node_final[['지하철역코드', '지하철역명', '경도', '위도']]

t_node_final.head()

지하철역코드     298
지하철역명      298
경도        3492
위도        3492
dtype: int64


,지하철역코드,지하철역명,경도,위도
0,1,압구정,127.028405,37.526718
10,2,학여울,127.070709,37.496559
13,3,삼성중앙,127.053294,37.513041
28,4,구룡,127.059775,37.486632
39,5,대모산입구,127.072832,37.491386


### 링크값을 가지는 역 처리

In [10]:
train_link = train[train['링크 WKT'].notnull()]
#train_link.info()
train_link.drop('노드 WKT', axis=1, inplace=True)
train_link.head()

C:\Users\User\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,링크 WKT,지하철역코드,지하철역명
0,127.00191146253691 37.5711642556191,266,종로5가
1,126.99214130747008 37.5704173818590,265,종로3가
2,127.00193818725627 37.5708237971757,266,종로5가
17,126.99116601227134 37.5725885933799,265,종로3가
25,127.00191146253691 37.5711642556191,266,종로5가


In [11]:
#경도 위도 나누기
tt = train_link['링크 WKT'].str.split(' ', expand=True)
tt.columns = ['경도', '위도']
tt = tt.astype(np.float)
tt.head()

,경도,위도
0,127.001911,37.571164
1,126.992141,37.570417
2,127.001938,37.570824
17,126.991166,37.572589
25,127.001911,37.571164


In [12]:
#합치기
ttt = pd.concat([train_link[['지하철역코드', '지하철역명']], tt], axis=1)

#unique한 값 개수 확인
print(ttt.nunique())

# 한 지하철 역에 여러개의 네트워크 기기가 설치된 곳이 많음, 위도경도 통일 필요 >> 평균 사용
tttt = ttt.groupby(by = '지하철역코드').mean()

#merge
t_link_final = pd.merge(right= train_link[['지하철역코드', '지하철역명']], left=tttt, how = 'left', on='지하철역코드')
t_link_final = t_link_final.drop_duplicates()
t_link_final = t_link_final[['지하철역코드', '지하철역명', '경도', '위도']]

t_link_final.head()

지하철역코드     290
지하철역명      290
경도        1906
위도        1906
dtype: int64


,지하철역코드,지하철역명,경도,위도
0,1,압구정,127.028310,37.526826
9,2,학여울,127.070758,37.496758
11,3,삼성중앙,127.053331,37.513085
25,4,구룡,127.059744,37.486683
35,5,대모산입구,127.072716,37.491375


### 나눴던거 합치기

In [24]:
#링크, 노드 합치기
train_concat = pd.concat([t_link_final, t_node_final], axis=0, ignore_index=True)

#unique한 값 개수 확인
train_concat.nunique()

# 한 정거장 당 2개씩 위치좌표가 반환됨, 통일 필요 >> 평균사용
train_group = train_concat.groupby(by=['지하철역코드', '지하철역명']).mean()
train_group.reset_index(level=['지하철역코드', '지하철역명'], inplace=True)

train_group.head()

,지하철역코드,지하철역명,경도,위도
0,1,압구정,127.028357,37.526772
1,2,학여울,127.070733,37.496658
2,3,삼성중앙,127.053312,37.513063
3,4,구룡,127.059759,37.486657
4,5,대모산입구,127.072774,37.491381


### 버스데이터 정제

In [27]:
bus['지하철역 수'] = 0
bus.head()

,정류소ID,ARS-ID,정류소명,X좌표,Y좌표,지하철역 수
0,100000001,1001,종로2가사거리,126.987750,37.569765,0
1,100000002,1002,창경궁.서울대학교병원,126.996566,37.579183,0
2,100000003,1003,명륜3가.성대입구,126.998340,37.582671,0
3,100000004,1004,종로2가.삼일교,126.987613,37.568579,0
4,100000005,1005,혜화동로터리.여운형활동터,127.001744,37.586243,0


In [30]:
bus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11703 entries, 0 to 11702
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   정류소ID   11703 non-null  int64  
 1   ARS-ID  11703 non-null  int64  
 2   정류소명    11703 non-null  object 
 3   X좌표     11703 non-null  float64
 4   Y좌표     11703 non-null  float64
 5   지하철역 수  11703 non-null  int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 548.7+ KB


In [35]:
bus.describe()

,정류소ID,ARS-ID,X좌표,Y좌표,지하철역 수
count,1.170300e+04,11703.000000,11703.000000,11703.000000,11703.000000
mean,1.205358e+08,14488.584295,126.982772,37.544705,0.662736
std,2.738363e+07,6957.472576,0.094118,0.071952,0.716759
min,1.000000e+08,1001.000000,126.696248,36.996806,0.000000
25%,1.080000e+08,8892.500000,126.912244,37.498675,0.000000
50%,1.140002e+08,14597.000000,126.991360,37.548278,1.000000
75%,1.209001e+08,20860.000000,127.052762,37.591805,1.000000
max,2.330019e+08,25999.000000,127.348658,37.935107,4.000000


In [31]:
print(bus['정류소ID'].nunique())
print(bus['ARS-ID'].nunique())
print(bus['정류소명'].nunique())

#나중에 문제되면 처리해야쥐~~~~

11703
11287
7501


### 버스정류장 근처 지하철 역 수 계산

In [34]:
for i in range(len(bus)):
    place1 = (bus.iloc[i, 3], bus.iloc[i, 4])
    for j in range(len(train_group)):
        place2 = (train_group.iloc[j, 2], train_group.iloc[j, 3])
        if haversine(place1, place2, unit='m') <500:
            bus.iloc[i,5] +=1

In [36]:
bus.head()

,정류소ID,ARS-ID,정류소명,X좌표,Y좌표,지하철역 수
0,100000001,1001,종로2가사거리,126.987750,37.569765,3
1,100000002,1002,창경궁.서울대학교병원,126.996566,37.579183,0
2,100000003,1003,명륜3가.성대입구,126.998340,37.582671,1
3,100000004,1004,종로2가.삼일교,126.987613,37.568579,3
4,100000005,1005,혜화동로터리.여운형활동터,127.001744,37.586243,1


In [39]:
s_train_final = bus[['정류소ID', '정류소명', '지하철역 수']]
s_train_final.head()

,정류소ID,정류소명,지하철역 수
0,100000001,종로2가사거리,3
1,100000002,창경궁.서울대학교병원,0
2,100000003,명륜3가.성대입구,1
3,100000004,종로2가.삼일교,3
4,100000005,혜화동로터리.여운형활동터,1


### csv로 저장

In [40]:
s_train_final.to_csv('서울 지하철역수.csv', index=False, encoding='cp949')

In [41]:
daramji = pd.read_csv("./따람쮜 버스 출퇴근 승껚.csv", encoding='cp949')
daramji.head()

,노선번호,노선명,표준버스정류장ID,버스정류장ARS번호,역명,6시승차총승객수,6시하차총승객수,7시승차총승객수,7시하차총승객수,8시승차총승객수,...,17시승차총승객수,17시하차총승객수,18시승차총승객수,18시하차총승객수,19시승차총승객수,19시하차총승객수,출근 승차,출근 하차,퇴근 승차,퇴근 하차
0,8221,8221번(장안2동주민센터~답십리역),105000059,6145,전곡시장앞,518,1,9135,25,8377,...,0,0,0,0,0,0,18287,68,0,0
1,8221,8221번(장안2동주민센터~답십리역),105000061,6147,답십리촬영소.우성그린아파트,281,2,8889,106,8811,...,0,0,0,0,0,0,18920,286,0,0
2,8221,8221번(장안2동주민센터~답십리역),105000064,6150,동답한신동아아파트,99,1,8939,137,9297,...,0,0,0,0,0,0,19189,502,0,0
3,8221,8221번(장안2동주민센터~답십리역),105000067,6153,동아.위더스빌아파트,2,0,4337,552,3711,...,0,0,0,0,0,0,8606,1417,0,0
4,8221,8221번(장안2동주민센터~답십리역),105000071,6157,고미술상가,0,2,48,5559,138,...,0,0,0,0,0,0,203,13474,0,0


In [44]:
dada = daramji['표준버스정류장ID'].tolist()

In [49]:
con = s_train_final['정류소ID'].isin(dada)
dadada = s_train_final[con]

dadada.describe()

,정류소ID,지하철역 수
count,9.400000e+01,94.000000
mean,1.165640e+08,0.606383
std,6.217532e+06,0.721678
min,1.050001e+08,0.000000
25%,1.110001e+08,0.000000
50%,1.190001e+08,0.000000
75%,1.200004e+08,1.000000
max,1.230006e+08,2.000000
